**The authors of QA-LoRA removed their code from GitHub. I used it to implement this notebook that is consequently not running anymore.**


This notebook fine-tunes QA-LoRA adapters for Llama 2. You need a Llama 2 model quantized with a specific version of AutoGPTQ. I provide one or you can quantize one by yourself by running the code in the appendix.
More details about QA-LoRA in this article: [Fine-tune Quantized Llama 2 on Your GPU with QA-LoRA](https://kaitchup.substack.com/p/fine-tune-quantized-llama-2-on-your)

Install a specific branch of AutoGPTQ and all the depencies of QA-LoRA.

I applied a patch that I made for Llama 2 support. It may not work anymore as QA-LoRA gets updated. Try first without the patch.

In [ ]:
!git clone -b v0.3.0 https://github.com/PanQiWei/AutoGPTQ.git
!git clone https://github.com/yuhuixu1993/qa-lora.git
%cd qa-lora
!pip install -r requirements.txt
%cd ..
!cp qa-lora/peft_utils.py ./AutoGPTQ/auto_gptq/utils/
!wget https://about.benjaminmarie.com/data/py/qalora/qalora.py
!cp qalora.py qa-lora/
%cd AutoGPTQ

!pip install .[triton]
%cd ..
!git clone https://github.com/timdettmers/bitsandbytes.git
%cd bitsandbytes
# CUDA_VERSIONS in {110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 120}
# make argument in {cuda110, cuda11x, cuda12x}
# if you do not know what CUDA you have, try looking at the output of: python -m bitsandbytes
!CUDA_VERSION=118 make cuda11x
!python setup.py install
!pip install -r requirements.txt
!pip install protobuf==3.20.*


#!pip install -r requirements.txt

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3541, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 3541 (delta 536), reused 591 (delta 509), pack-reused 2845
Receiving objects: 100% (3541/3541), 7.75 MiB | 16.09 MiB/s, done.
Resolving deltas: 100% (2329/2329), done.
Note: switching to '45576f0933f5e9ef7c1617006d5db359e1669155'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Cloning into 'qa-lora'...
remote: Enumerating objects: 118, done.
remot

Fine-tune Llama 2 with QA-LoRA on Alpaca for 100 steps. The checkpoints are saved in the directory output/

In [ ]:
%cd /content/qa-lora/
!python qalora.py --model_path kaitchup/Llama-2-7b-4bit-32g-autogptq \
                  --save_steps 10 \
                  --output_dir output \
                  --per_device_train_batch_size 1 \
                  --gradient_accumulation_steps 16 \
                  --max_steps 100 \
                  --lora_r 16 \


/content/qa-lora
2023-10-11 18:23:09.848829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading base model kaitchup/Llama-2-7b-4bit-32g-autogptq...
adding LoRA modules...
/usr/local/lib/python3.10/dist-packages/auto_gptq/utils/peft_utils.py:369: UserWarning: You can just ignore this warning if the peft type you use isn't in ['LORA', 'ADALORA'].
LlamaGPTQForCausalLM supports injecting fused attention but not enables this time. If you are training adapters, you must also disable fused attention injection when loading quantized base model at inference time, otherwise adapters may not be added to base model properly. If you are loading adapters to do inference, you can reference to adapter's config file to check whether the adapters are trained using base model that not enable fused attention injection.
  warnings.warn(
trainable params: 11163648.0 || all params: 1162579968 || trainable: 0.960247751318557
loaded model
torch.float16 26377

Merge the QA-LoRA adapter with the base model. The base model and the adapter must be accessible locally.

In [ ]:
import torch

#Path the quantized base model
model_path = 'Llama-2-7b-4bit-32g-autogptq/gptq_model-4bit-32g.bin'
#Path to the adapter fine-tuned with QA-LoRA
lora_path = 'output/checkpoint-100/adapter_model.bin'
#Where the merged model will be saved
merged_path = 'output_model'
#The scale is the LoRA alpha divided by the LoRA rank. I trained with LoRA_alpha = LoRA_rank = 16
scale = 16 / 16
#The group size of the quantized base LLN
group_size = 32

#We merge using the CPU
model = torch.load(model_path, map_location='cpu')
lora = torch.load(lora_path, map_location='cpu')
tmp_keys = [key[17:-14] for key in lora.keys() if 'lora_A' in key]
for tmp_key in tmp_keys:
    model[tmp_key+'.qzeros'] -= (lora['base_model.model.'+tmp_key+'.lora_B.weight'] @ lora['base_model.model.'+tmp_key+'.lora_A.weight']).t() * scale / group_size /model[tmp_key+'.scales']

torch.save(model, merged_path)


# Appendix: Quantize with AutoGPTQ

More details here: [GPTQ or bitsandbytes: Which Quantization Method to Use for LLMs — Examples with Llama 2
](https://kaitchup.substack.com/p/gptq-or-bitsandbytes-which-quantization)

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import random
import numpy as np
import torch
#The model we want to quantize
pretrained_model_dir = "meta-llama/Llama-2-7b-hf"

#The name of the model once quantized
#Note that we will only save the model, the tokenizer will remain the same
quantized_model_dir = "Llama-2-7b-4bit-32g-hf"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_dir, use_fast=True)
#I copied and edited this function from AutoGPTQ repository
def get_wikitext2(nsamples, seed, seqlen, model):

    #I load the validation split since the training split is unecessary large
    traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='validation')


    trainenc = tokenizer("\n\n".join(traindata['text']), return_tensors='pt')

    random.seed(seed)
    np.random.seed(0)
    torch.random.manual_seed(0)

    traindataset = []

    #This is unecessary to use the entire dataset for calibration
    #Here I use only 128 samples
    for _ in range(nsamples):
        i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        inp = trainenc.input_ids[:, i:j]
        attention_mask = torch.ones_like(inp)
        traindataset.append({'input_ids':inp,'attention_mask': attention_mask})
    return traindataset

#Process the data that will be used for calibration
traindataset = get_wikitext2(128, 0, 2048, pretrained_model_dir)


quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=32,
    desc_act=False,
)


# load Llama 2
model = AutoGPTQForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)

# quantize model, using traindataset for calibration
#this may take up to 10 minutes on Google Colab Pro
model.quantize(traindataset, use_triton=True)

# save quantized model using safetensors
model.save_quantized(quantized_model_dir,  use_safetensors=False)
model.push_to_hub("kaitchup/Llama-2-7b-4bit-32g-autogptq", use_safetensors=False)
tokenizer.push_to_hub("kaitchup/Llama-2-7b-4bit-32g-autogptq")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

gptq_model-4bit-32g.bin:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


CommitInfo(commit_url='https://huggingface.co/kaitchup/Llama-2-7b-4bit-32g-autogptq/commit/40567429f69fadb7f235abe6b236e07abd8ddfbf', commit_message='Upload tokenizer', commit_description='', oid='40567429f69fadb7f235abe6b236e07abd8ddfbf', pr_url=None, pr_revision=None, pr_num=None)